In [27]:
from scipy.sparse import csr_matrix
import sparse
import numpy as np
import pyamg
import scipy.sparse

In [28]:
class mycsr(csr_matrix):
    """
    *OpenMP Enabled*
    """
    def _mul_vector(self, other):        
        M,N = self.shape

        # output array
        result = np.zeros(M, dtype=scipy.sparse.sputils.upcast_char(self.dtype.char,
                                                                    other.dtype.char))

        # csr_matvec or csc_matvec
        sparse.csr_matvec_omp(M, N, self.indptr, self.indices, self.data, other, result)

        return result

mycsr.__doc__ += csr_matrix.__doc__

In [29]:
from timeit import default_timer as timer

In [30]:
A = pyamg.gallery.poisson((3,3), format='csr')
v = np.random.rand(A.shape[0])

In [31]:
Amycsr = mycsr(A)

In [32]:
w1 = A * v

In [33]:
w2 = Amycsr * v

In [34]:
np.testing.assert_array_almost_equal(w1, w2)

In [10]:
Amycsr?

In [40]:
np.random.seed(555)
nx = 100
A = pyamg.gallery.poisson((nx, nx, nx), format='csr')
b = np.random.rand(A.shape[0])

x0 = np.random.rand(A.shape[0])

In [41]:
ml = pyamg.smoothed_aggregation_solver(A, max_coarse=10)

In [42]:
t0 = timer()
x = ml.solve(b, x0=x0, maxiter=10)
t1 = timer()
print(t1-t0, ' seconds')

2.3374479159992916  seconds


In [43]:
for i in range(len(ml.levels)):
    ml.levels[i].A = mycsr(ml.levels[i].A)

In [44]:
t0 = timer()
x = ml.solve(b, x0=x0, maxiter=10)
t1 = timer()
print(t1-t0, ' seconds')

1.419024009000168  seconds
